# Testing the lineup plus-minus

In [1]:
import sys
import logging

from nba_survival.data.endpoints import GameRotation, PlayByPlay, TeamLineups
from nba_survival.data.tasks import get_survival_time, get_lineup_rating

logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

In [2]:
rotation = GameRotation(GameID="0021800006", output_dir="../nba-data/2018-19")
awaylineup = TeamLineups(TeamID=1610612748, output_dir="../nba-data/2018-19")
homelineup = TeamLineups(TeamID=1610612753, output_dir="../nba-data/2018-19")
pbp = PlayByPlay(GameID="0021800006", output_dir="../nba-data/2018-19")

In [3]:
rotation.load()
homelineup.load()
awaylineup.load()
pbp.load()

INFO:nba_survival.data.endpoints.base:Reading existing file ../nba-data/2018-19/gamerotation/data_0021800006.json...
INFO:nba_survival.data.endpoints.base:Reading existing file ../nba-data/2018-19/teamdashlineups/data_1610612753.json...
INFO:nba_survival.data.endpoints.base:Reading existing file ../nba-data/2018-19/teamdashlineups/data_1610612748.json...
INFO:nba_survival.data.endpoints.base:Reading existing file ../nba-data/2018-19/playbyplayv2/data_0021800006.json...


In [4]:
pbp_df = pbp.get_data()
away_rotation = rotation.get_data("AwayTeam")
home_rotation = rotation.get_data("HomeTeam")
homelineupstats = homelineup.get_data("Lineups")
awaylineupstats = awaylineup.get_data("Lineups")

In [5]:
away_rotation.sort_values(by=["IN_TIME_REAL", "OUT_TIME_REAL"], ascending=True, inplace=True)

away_rotation.head(n=10)

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
20,0021800006,1610612748,Miami,Heat,203585,Rodney,McGruder,0.0,2180.0,0,5.0,0.125
30,0021800006,1610612748,Miami,Heat,1626196,Josh,Richardson,0.0,3000.0,2,6.0,0.182
4,0021800006,1610612748,Miami,Heat,201609,Goran,Dragic,0.0,5350.0,10,11.0,0.227
10,0021800006,1610612748,Miami,Heat,202355,Hassan,Whiteside,0.0,5350.0,4,11.0,0.227
35,0021800006,1610612748,Miami,Heat,1627884,Derrick,Jones Jr.,0.0,5350.0,5,11.0,0.182
0,0021800006,1610612748,Miami,Heat,2548,Dwyane,Wade,2180.0,8560.0,2,-3.0,0.250
25,0021800006,1610612748,Miami,Heat,204020,Tyler,Johnson,3000.0,9900.0,5,-4.0,0.152
31,0021800006,1610612748,Miami,Heat,1626196,Josh,Richardson,5350.0,5900.0,3,0.0,0.500
38,0021800006,1610612748,Miami,Heat,1628389,Bam,Adebayo,5350.0,8560.0,0,-9.0,0.200
15,0021800006,1610612748,Miami,Heat,203482,Kelly,Olynyk,5350.0,10480.0,0,-10.0,0.080


In [6]:
pbp_df = get_survival_time(pbp_df)

In [7]:
pbp_df = get_lineup_rating(
    pbp=pbp_df,
    home_lineup_stats=homelineupstats,
    away_lineup_stats=awaylineupstats,
    home_rotation=home_rotation,
    away_rotation=away_rotation
)

INFO:nba_survival.data.tasks:Adding starting lineup plus minus for 0021800006
INFO:nba_survival.data.tasks:Looping through each event in game 0021800006
INFO:nba_survival.data.tasks:Looking for substitutions at the beginning of 1...
INFO:nba_survival.data.tasks:New home team players at the start of period 1
INFO:nba_survival.data.tasks:Substituting the following players in: 201571, 202696, 203095, 203932, 1628371
DEBUG:nba_survival.data.tasks:Looking for the following lineup group: 1628371-201571-202696-203095-203932
INFO:nba_survival.data.tasks:New visiting team  players at the start of period 1
INFO:nba_survival.data.tasks:Substituting the following players in: 203585, 1626196, 201609, 202355, 1627884
DEBUG:nba_survival.data.tasks:Looking for the following lineup group: 1626196-1627884-201609-202355-203585
INFO:nba_survival.data.tasks:At 8:22 in period 1: SUB: Wade FOR McGruder
DEBUG:nba_survival.data.tasks:Removing 203585
DEBUG:nba_survival.data.tasks:Adding 2548
DEBUG:nba_survival.

In [10]:
pbp_df.loc[
    pbp_df["EVENTMSGTYPE"] == 8,
    [
        "TIME",
        "PERIOD",
        "PCTIMESTRING",
        "SCOREMARGIN",
        "HOMEDESCRIPTION",
        "VISITORDESCRIPTION",
        "HOME_LINEUP_PLUS_MINUS",
        "VISITOR_LINEUP_PLUS_MINUS"
    ]
].head(n=50)

,TIME,PERIOD,PCTIMESTRING,SCOREMARGIN,HOMEDESCRIPTION,VISITORDESCRIPTION,HOME_LINEUP_PLUS_MINUS,VISITOR_LINEUP_PLUS_MINUS
36,218,1,8:22,None,None,SUB: Wade FOR McGruder,0.7,-1.2
51,300,1,7:00,None,None,SUB: Johnson FOR Richardson,0.7,4.0
63,388,1,5:32,None,SUB: Bamba FOR Gordon,None,0.0,4.0
64,388,1,5:32,None,SUB: Ross FOR Isaac,None,-0.5,4.0
87,535,1,3:05,None,SUB: Simmons FOR Vucevic,None,0.0,4.0
88,535,1,3:05,None,SUB: Grant FOR Fournier,None,0.0,4.0
89,535,1,3:05,None,SUB: Gordon FOR Augustin,None,-0.7,4.0
90,535,1,3:05,None,None,SUB: Adebayo FOR Jones Jr.,-0.7,0.0
91,535,1,3:05,None,None,SUB: Richardson FOR Dragic,-0.7,0.0
92,535,1,3:05,None,None,SUB: Olynyk FOR Whiteside,-0.7,0.5
